In [1]:
%%bash
date

########################################
##### need to specify ################## 
projPath=/path/to/save/project/
cores=4

histNames=(C_H3K27me3 \
           C_H3K4me3 \
           C_IgG) 

histNames2=(C_H3K27me3 \
           C_H3K4me3) 

# Declare an associative array
declare -A hist_control_map=(["C_H3K27me3"]="C_IgG" \
                            ["C_H3K4me3"]="C_IgG")
########################################
########################################

##################################################################
## Step 1. Preprocess files
##################################################################

inputPath=${projPath}/processed_data/CUT_RUN/Part1_alignment/
inputPath2=${projPath}/processed_data/CUT_RUN/Part2_ecoli_alignment/
outPath=${projPath}/processed_data/CUT_RUN/Part3_peak_calling/
my_genome=${projPath}/meta/cut_run/sizes.genome

## mkdir for output files
mkdir -p $outPath/Step1_sortBam/sort_wo_n
mkdir -p $outPath/Step2_bam2bedgraph/
mkdir -p $outPath/Step3_peak_calling/

for histName in "${histNames[@]}"
do
 
    scale_factor=$(sed -n '3p' $inputPath2/alignment/sam/bowtie2_summary/${histName}_bowtie2_scale_factor.txt | awk '{print $1}')
    
    bedtools bamtobed -bedpe -i $inputPath/alignment/bam/${histName}_bowtie2.mapped.bam \
        > $outPath/Step2_bam2bedgraph/${histName}.bed \
        2> $outPath/Step2_bam2bedgraph/${histName}.bed.log

    awk '$1==$4 && $6-$2 < 1000 {print $0}' \
        $outPath/Step2_bam2bedgraph/${histName}.bed \
        > $outPath/Step2_bam2bedgraph/${histName}.clean.bed \
        2> $outPath/Step2_bam2bedgraph/${histName}.clean.bed.log

    cut -f 1,2,6 $outPath/Step2_bam2bedgraph/${histName}.clean.bed | sort -k1,1 -k2,2n -k3,3n \
        > $outPath/Step2_bam2bedgraph/${histName}.fragments.bed

    bedtools genomecov -bg -scale ${scale_factor} \
        -i $outPath/Step2_bam2bedgraph/${histName}.fragments.bed \
        -g $my_genome > $outPath/Step2_bam2bedgraph/${histName}.fragments.bedgraph \
        2> $outPath/Step2_bam2bedgraph/${histName}.fragments.bedgraph.log
    
    ## remove intermediate files
    rm $outPath/Step2_bam2bedgraph/${histName}.fragments.bed
    rm $outPath/Step2_bam2bedgraph/${histName}.clean.bed
    rm $outPath/Step2_bam2bedgraph/${histName}.bed
    
done

##################################################################
## Step 2. Call peaks with SEACR
##################################################################

seacr=${projPath}/code/SEACR_1.3.sh

for histName in "${histNames2[@]}"
do
    controlName=${hist_control_map["${histName}"]}
    
    bash ${seacr} ${outPath}/Step2_bam2bedgraph/${histName}.fragments.bedgraph \
     ${outPath}/Step2_bam2bedgraph/${controlName}.fragments.bedgraph \
     non stringent ${outPath}/Step3_peak_calling/${histName}_seacr_${controlName}.peaks \
     &> ${outPath}/Step3_peak_calling/${histName}_seacr_${controlName}.peaks.log
done


date

Thu 18 Jan 2024 12:01:08 AM CST
Thu 18 Jan 2024 01:37:51 AM CST
